## Rocchioを試す

### 調べる

**論文**

http://db-event.jpn.org/deim2013//proceedings/pdf/b3-1.pdf


#### 手法概要

コサイン類似度をもとにしている
ユーザーが適合と判断した文書に含まれる語の重みをTFIDFなどで大きくし、不適合を逆に小さくすることで、クエリベクトルを修正する

クエリと単語の関連度を表す指標として、Bhattacharyya係数に共起度の逆数をかけたBCiqを使う


#### フィードバックについて

ユーザーがクエリを与えたとき関連度(BC)を元に得た適合文書の上位N件を使う
ユーザーが閲覧したものを適合、していないものを不適合として関連度を更新していく

### 実際の処理手順

- まず普通にユーザーからのクエリを元に関連文書の上位N件をユーザーに提示する
- ユーザーにその中から選ばせる。
- リランキングする
- ユーザーに精度の向上した選択肢を提示できる

これはどうやらRocchioではないらしい、まずはフィードバックのアルゴリズムとしてポピュラーなRocchioを試してみる


## Rocchioを改めて試す

### 調べる

コーセラでわかりやすい動画があった

https://www.coursera.org/learn/text-retrieval/lecture/PyTkW/lesson-5-2-feedback-in-vector-space-model-rocchio

これによると、かなり単純なアルゴリズムだった。

![rocchio](assets/rocchio.png)

`新しいクエリ = (クエリに重みを掛けたもの) + (ポジティブドキュメントの中央値に重みをかけたもの)　- (ネガティブドキュメントの中央値に重みをかけたもの)`

## 試す

In [1]:
from IPython.core.display import display

# ポジティブドキュメント
D3 = np.array((1.5, 0, 3.0, 2.0, 0, 0))
D4 = np.array((1.5, 0, 4.0, 2.0, 0, 0))
display(D3)
display(D4)

# ネガティブドキュメント
D1 = np.array((1.5, 0.1, 0, 0, 0, 0))
D2 = np.array((1.5, 0.1, 0, 2.0, 2.0, 0))
D5 = (1.5, 0, 0, 6.0, 2.0, 0)
display(D1)
display(D2)
display(D5)

# ポジティブドキュメントの中心
positive_centroid_vector = (D3 + D4) / 2
display(positive_centroid_vector)

negative_centroid_vector = (D1 + D2 + D5) / 3
display(negative_centroid_vector)

original_query = np.array((1, 1, 1, 1, 0, 0))

# ハイパーパラメーター
alpha = 1
beta = 1
gamma = 1

new_query = (alpha * original_query) + (beta * positive_centroid_vector) - (gamma * negative_centroid_vector)
display(new_query)

# 上記のままだとパフォーマンスに問題があるので、ネガティブドキュメントから一番高いいくつかのベクトルのみを抽出するほうがいいらしい


array([ 1.5,  0. ,  3. ,  2. ,  0. ,  0. ])

array([ 1.5,  0. ,  4. ,  2. ,  0. ,  0. ])

array([ 1.5,  0.1,  0. ,  0. ,  0. ,  0. ])

array([ 1.5,  0.1,  0. ,  2. ,  2. ,  0. ])

(1.5, 0, 0, 6.0, 2.0, 0)

array([ 1.5,  0. ,  3.5,  2. ,  0. ,  0. ])

array([ 1.5       ,  0.06666667,  0.        ,  2.66666667,  1.33333333,  0.        ])

array([ 1.        ,  0.93333333,  4.5       ,  0.33333333, -1.33333333,  0.        ])

## sklearnを使って試す

In [2]:
'''
    learningディレクトリのパスを取得／設定
'''
import sys
import os

if os.path.basename(os.getcwd()) != 'learning':
    learning_dir = os.path.join(os.getcwd(), '../..')
    learning_dir = os.path.abspath(learning_dir)
    os.chdir(learning_dir)

    if learning_dir not in sys.path:
        sys.path.append(learning_dir)

print('current_dir={}'.format(os.getcwd()))


'''
    ライブラリロード
'''
from IPython.core.display import display

'''
   初期化
'''
from tests.support.helper import Helper
Helper.init()

current_dir=/Users/shwld/project/mofmof/donusagi-bot/learning


/Users/shwld/.pyenv/versions/3.5.2/envs/donusagi-bot3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/shwld/.pyenv/versions/3.5.2/envs/donusagi-bot3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from app.shared.datasource.file.ratings import Ratings
from app.core.tokenizer.mecab_tokenizer import MecabTokenizer
from app.core.vectorizer.tfidf_vectorizer import TfidfVectorizer
from sklearn.neighbors.nearest_centroid import NearestCentroid

tokenizer = MecabTokenizer.new()
vectorizer = TfidfVectorizer.new()

[DEBUG 140736093508416 2017/11/27 PM 12:41:26  mecab_tokenizer_with_split: 10 -             __init__] dicdir: /usr/local/lib/mecab/dic/mecab-ipadic-neologd


## コサイン類似検索で普通に解答を取得してみる
あとで補正された解答と比較するため

In [4]:
# クエリドキュメント(question)のベクトルを取得する

question = tokenizer.tokenize(['らく賃について聞きたいのですが誰が担当なのかわかりますか?'])
q_vectors = vectorizer.transform(question)
display(q_vectors.shape)

(1, 894)

In [5]:
from app.core.cosine_similarity import CosineSimilarity
from app.shared.constants import Constants

context = Helper.test_context(bot_id=30, algorithm=Constants.ALGORITHM_SIMILARITY_CLASSIFICATION)
cs = CosineSimilarity.new(bot=context.current_bot, vectorizer=vectorizer)
data_frame = cs.predict(q_vectors)
data_frame = data_frame.drop_duplicates(subset='question_answer_id', keep='first')
data_frame = data_frame.sort_values(by='probability', ascending=False)
data_frame[:10]

[ INFO 140736093508416 2017/11/27 PM 12:41:29                     context: 21 -             __init__] algorithm: Simmilarity Classification
[DEBUG 140736093508416 2017/11/27 PM 12:41:29  mecab_tokenizer_with_split: 10 -             __init__] dicdir: /usr/local/lib/mecab/dic/mecab-ipadic-neologd


,question,question_answer_id,probability
10860,救急箱の担当は誰。,24628,0.404455
10859,らく賃の担当者は誰。\n,24627,0.403036
10674,ちょっと聞きたい,23903,0.387277
10617,わからない,23844,0.379846
10720,保険の代理店監査の担当は誰。\n\n,24481,0.303962
10693,らく賃を受託,24440,0.241121
10791,周辺商品でらく賃（受託）の担当者は誰。,24558,0.240588
10792,周辺商品でらく賃（申込）の担当者は誰。,24559,0.238946
10870,パスWordがわからなくなった。,24642,0.229104
10546,今期（第20期、2017年）、社員旅行が中止と聞きましたが、出勤ですか。,23773,0.204969


救急箱の担当という全く関係ない解答が一番に来ている。これを下げたい

## Rocchioでクエリを補正して再度解答を得る

In [6]:
# ポジティブドキュメント(good ratings)のベクトルを取得する

positive_docs = [
    'らく賃の担当者は誰。\n',
    '周辺商品でらく賃（受託）の担当者は誰。',
    '周辺商品でらく賃（申込）の担当者は誰。',
]
posi_sentences = tokenizer.tokenize(positive_docs)
posi_vectors = vectorizer.transform(posi_sentences)
display(posi_vectors.shape)

# 質問に一番近いドキュメントを取得する

posi_clf = NearestCentroid()
posi_clf.fit(posi_vectors, range(len(positive_docs)))

posi_index = posi_clf.predict(q_vectors)
display(positive_docs[posi_index])

(3, 894)

'らく賃の担当者は誰。\n'

In [7]:
# ネガティブドキュメント(bad ratings)のベクトルを取得する

negative_docs = [
    '救急箱の担当は誰。',
    'ちょっと聞きたい',
    '自社物の契約中の物件だが戸別らく賃に切り替えたいのですが、オリコの審査に落ちました。',
]
nega_sentences = tokenizer.tokenize(negative_docs)
nega_vectors = vectorizer.transform(nega_sentences)
display(nega_vectors.shape)


# 質問に一番近いratingsを取得する

nega_clf = NearestCentroid()
nega_clf.fit(posi_vectors, range(len(negative_docs)))

nega_index = nega_clf.predict(q_vectors)
display(negative_docs[nega_index])

(3, 894)

'救急箱の担当は誰。'

In [8]:
# Rocchioで補正されたクエリベクタを取得する

# Note: ハイパーパラメータ
alpha = 1 # クエリの重み　一番高くしたい
beta = 0.5# ポジティブドキュメントの重み
gamma = 0.3 # ネガティブドキュメントの重み 低くしたい

new_q_vectors = (alpha * q_vectors) + (beta * posi_vectors[posi_index]) - (gamma * nega_vectors[nega_index])
display(new_q_vectors.shape)

(1, 894)

### 補正されたベクトルで解答を取得して比較してみる

In [9]:
rocchio_res = cs.predict(new_q_vectors)
rocchio_res = rocchio_res.drop_duplicates(subset='question_answer_id', keep='first')
rocchio_res = rocchio_res.sort_values(by='probability', ascending=False)
rocchio_res[:10]


,question,question_answer_id,probability
10859,らく賃の担当者は誰。\n,24627,0.706313
10791,周辺商品でらく賃（受託）の担当者は誰。,24558,0.421626
10792,周辺商品でらく賃（申込）の担当者は誰。,24559,0.418749
10693,らく賃を受託,24440,0.383253
10674,ちょっと聞きたい,23903,0.321955
10617,わからない,23844,0.315778
10454,らく賃の客付けをしました。らく賃の申込時の処理を知りたい。,23675,0.303831
10487,定期借家契約の物件だがらく賃・全室らく賃・らく賃サポート・トータルらく賃で受託はできるのか。,23714,0.285281
10486,定期借家契約の物件だがらく賃・全室らく賃・らく賃サポート・トータルらく賃に切り替えはできるのか。,23713,0.279907
10692,らく賃の申込を取得しました。,24439,0.268616


思った通りに上がった!

ハイパーパラメータの調整で影響も調整できそうなのでかなり良さげ